# Spin Qubit Pulse Sequences with the HDAWG

This notebook shows you how to use pulse sequences with an HDAWG to realize various experiments. While the sequences are mostly general, they are adapted to those typically used for spin qubits by adding the gate pulsing to control between Coulomb and spin blockade.

List of experiemnts
* Rabi - sweep length of burst
* Ramsey variant 1 - sweep delay with constant burst duration
* Ramsey variant 2 - sweep burst duration at constant delay
* Ramsey variant 3 - sweep phase of second burst and delay between bursts

# General Imports

In [2]:
%config IPCompleter.greedy=True

import matplotlib.pyplot as plt
import numpy as np

from laboneq.simple import *

# Device Setup

## Calibration

In [3]:
def calibrate_devices(device_setup):

    device_setup.logical_signal_groups["q0"].logical_signals[
        "drive_line"
    ].calibration = SignalCalibration(
        oscillator=Oscillator(
            uid="drive_q0_osc", frequency=1e8, modulation_type=ModulationType.HARDWARE
        ),
        mixer_calibration=MixerCalibration(
            voltage_offsets=[0.0, 0.0],
            correction_matrix=[
                [1.0, 0.0],
                [0.0, 1.0],
            ],
        ),
    )


## Create device setup

In [4]:
descriptor = f"""\
instrument_list:
  HDAWG:
  - address: DEV8431
    uid: device_hdawg
connections:
  device_hdawg:    
    - iq_signal: q0/drive_line
      ports: [SIGOUTS/2, SIGOUTS/3]
    - rf_signal: q0/coulomb_line_1
      ports: [SIGOUTS/0]        
    - rf_signal: q0/coulomb_line_2
      ports: [SIGOUTS/1]   
"""

device_setup = DeviceSetup.from_descriptor(
    descriptor,
    server_host="your_ip_address",
    server_port=8004,
    setup_name="MySetup",
)
calibrate_devices(device_setup)


# Rabi Experiment: sweep burst length

## Define parameters for experiment

In [42]:
## constant definition
LEN_COULOMB_CYCLE = 200e-9
LEN_READOUT = 2 * LEN_COULOMB_CYCLE
X90_DURATION = 10e-9

START = 0
STOP = LEN_COULOMB_CYCLE / 2
STEPS = 5
NUM_REP = 2


In [43]:
## define length sweep parameter
length_sweep_parameter = LinearSweepParameter(
    uid="length_sweep", start=START, stop=STOP, count=STEPS
)


## Experiment

In [44]:
# define pulse shapes
coulomb_pulse = pulse_library.const(
    uid="coulomb_pulse", length=LEN_COULOMB_CYCLE / 2, amplitude=0.5
)
coulomb_readout = pulse_library.const(
    uid="coulomb_readout", length=LEN_READOUT, amplitude=1
)
drive_pulse = pulse_library.const(
    uid="rabi_drive_pulse", length=X90_DURATION, amplitude=1
)


In [45]:
## Create Experiment
exp = Experiment(
    "Rabi with Coulomb pulse",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("coulomb_1"),
        ExperimentSignal("coulomb_2"),
    ],
)

# define experiment
with exp.acquire_loop_rt(
    uid=("shots"), count=NUM_REP, averaging_mode=AveragingMode.SEQUENTIAL
):
    with exp.sweep(parameter=length_sweep_parameter):
        with exp.section(
            uid=("empty"),
            length=LEN_COULOMB_CYCLE / 2,
            alignment=SectionAlignment.RIGHT,
        ):
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.5)
            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.5)
        with exp.section(
            uid=("manipulation"),
            length=LEN_COULOMB_CYCLE / 2,
            alignment=SectionAlignment.RIGHT,
        ):
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.75)
            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.75)
            exp.play(signal="drive", pulse=drive_pulse, length=length_sweep_parameter)
        with exp.section(uid="qubit_readout", length=LEN_READOUT):
            exp.play(signal="coulomb_1", pulse=coulomb_readout, amplitude=0.3)
            exp.play(signal="coulomb_2", pulse=coulomb_readout, amplitude=0.3)
        with exp.section(
            uid="outer_trigger",
            length=LEN_READOUT,
            trigger={"drive": {"state": 1}},
            alignment=SectionAlignment.RIGHT,
        ):
            with exp.section(
                uid="inner_trigger",
                length=LEN_READOUT - 100e-9,
                trigger={"drive": {"state": 2}},
            ):
                exp.reserve(signal="drive")


In [46]:
# define signal maps for different qubits
map_q0 = {
    "drive": "/logical_signal_groups/q0/drive_line",
    "coulomb_1": "/logical_signal_groups/q0/coulomb_line_1",
    "coulomb_2": "/logical_signal_groups/q0/coulomb_line_2",
}

# calibration  for qubit 0
calib_q0 = Calibration()
calib_q0["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=500e6,
        modulation_type=ModulationType.HARDWARE,
    )
)


In [47]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=True)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)


2022.12.21 20:01:13.159 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is c:\Users\florianf\OneDrive - Zurich Instruments AG\Documents\QuantumPlaygrounds\florian-playground\LabOneQ_SpinQubits\laboneq_output\log
2022.12.21 20:01:13.161 laboneq.controller.controller  INFO   VERSION: laboneq 1.6.0a20221221135807+0cba0f4d
2022.12.21 20:01:13.162 laboneq.controller.devices.device_collection INFO   Connecting to data server at 10.42.11.2:8004
2022.12.21 20:01:13.164 laboneq.controller.communication INFO   Connected to Zurich Instrument's Data Server version 22.08 at 10.42.11.2:8004
2022.12.21 20:01:13.167 laboneq.controller.devices.device_collection INFO   Configuring clock sources
2022.12.21 20:01:13.169 laboneq.controller.devices.device_collection INFO   Clock sources configured


In [48]:
if not session.connection_state.emulated:
    instrument_serial = device_setup.instrument_by_uid("device_hdawg").address
    device = session.devices[instrument_serial]
    device.triggers.out[2].delay(23.9e-9)


In [49]:
session.run(exp)


2022.12.21 20:01:16.294 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2022.12.21 20:01:16.402 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 125
2022.12.21 20:01:16.403 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 4128
2022.12.21 20:01:16.404 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2022.12.21 20:01:16.455 laboneq.controller.controller  INFO   Starting near-time execution...
2022.12.21 20:01:16.466 laboneq.controller.controller  INFO   Finished near-time execution.


## View experiment in pulse sheet viewer

In [19]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Spin qubit length Rabi", compiled_exp)


2022.12.21 19:35:38.020 laboneq.pulse_sheet_viewer.pulse_sheet_viewer INFO   Writing html file to c:\Users\florianf\OneDrive - Zurich Instruments AG\Documents\QuantumPlaygrounds\florian-playground\LabOneQ_SpinQubits\Spin qubit length Rabi_2022-12-21-19-35-38.html


c:\Users\florianf\OneDrive - Zurich Instruments AG\Documents\QuantumPlaygrounds\florian-playground\LabOneQ_SpinQubits\Spin qubit length Rabi_2022-12-21-19-35-38.html

# Ramsey Experiment I: sweep wait time at constant burst length

## Experiment

In [56]:
START = 0
STOP = LEN_COULOMB_CYCLE / 2 - 2 * X90_DURATION
STEPS = 3
NUM_REP = 20


In [57]:
## Define sweep parameter
sweep_delay = LinearSweepParameter(
    uid="Ramsey_delay", start=START, stop=STOP, count=STEPS
)


In [58]:
## Create Experiment
exp = Experiment(
    "Ramsey variant I",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("coulomb_1"),
        ExperimentSignal("coulomb_2"),
    ],
)

# define experiment
with exp.acquire_loop_rt(
    uid="shots", count=NUM_REP, averaging_mode=AveragingMode.SEQUENTIAL
):
    with exp.sweep(uid="sweep", parameter=sweep_delay):
        with exp.section(
            uid="qubit_excitation",
            alignment=SectionAlignment.RIGHT,
            length=LEN_COULOMB_CYCLE,
        ):
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.5)
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.75)

            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.5)
            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.75)

            exp.play(signal="drive", pulse=drive_pulse)
            exp.delay(signal="drive", time=sweep_delay)
            exp.play(signal="drive", pulse=drive_pulse)
        with exp.section(uid="qubit_readout", length=LEN_READOUT):
            exp.play(signal="coulomb_1", pulse=coulomb_readout, amplitude=0.3)
            exp.play(signal="coulomb_2", pulse=coulomb_readout, amplitude=0.3)
            with exp.section(
                uid="outer_trigger",
                length=LEN_READOUT,
                trigger={"drive": {"state": 1}},
                alignment=SectionAlignment.RIGHT,
            ):
                with exp.section(
                    uid="inner_trigger",
                    length=LEN_READOUT - 100e-9,
                    trigger={"drive": {"state": 2}},
                ):
                    exp.reserve(signal="drive")


In [59]:
# define signal maps for different qubits
map_q0 = {
    "drive": "/logical_signal_groups/q0/drive_line",
    "coulomb_1": "/logical_signal_groups/q0/coulomb_line_1",
    "coulomb_2": "/logical_signal_groups/q0/coulomb_line_2",
}

# calibration  for qubit 0
calib_q0 = Calibration()
calib_q0["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=500e6,
        modulation_type=ModulationType.HARDWARE,
    )
)


In [62]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=False)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)

session.run(exp)


2022.12.21 20:05:16.532 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is c:\Users\florianf\OneDrive - Zurich Instruments AG\Documents\QuantumPlaygrounds\florian-playground\LabOneQ_SpinQubits\laboneq_output\log
2022.12.21 20:05:16.534 laboneq.controller.controller  INFO   VERSION: laboneq 1.6.0a20221221135807+0cba0f4d
2022.12.21 20:05:16.535 laboneq.controller.devices.device_collection INFO   Connecting to data server at 10.42.11.2:8004
2022.12.21 20:05:17.652 laboneq.controller.communication INFO   Connected to Zurich Instrument's Data Server version 22.08 at 10.42.11.2:8004
2022.12.21 20:05:25.119 laboneq.controller.devices.device_collection INFO   Configuring clock sources
2022.12.21 20:05:25.226 laboneq.controller.devices.device_collection INFO   Clock sources configured
2022.12.21 20:05:29.788 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2022.12.21 20:05:29.908 l

## View experiment in pulse sheet viewer

In [61]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Ramsey variant I", compiled_exp)


2022.12.21 20:02:09.568 laboneq.pulse_sheet_viewer.pulse_sheet_viewer INFO   Writing html file to c:\Users\florianf\OneDrive - Zurich Instruments AG\Documents\QuantumPlaygrounds\florian-playground\LabOneQ_SpinQubits\Ramsey variant I_2022-12-21-20-02-09.html


c:\Users\florianf\OneDrive - Zurich Instruments AG\Documents\QuantumPlaygrounds\florian-playground\LabOneQ_SpinQubits\Ramsey variant I_2022-12-21-20-02-09.html

# Ramsey Experiment II: sweep burst length at constant wait time

In [63]:
# define constant delay
T_DELAY = 40e-9


In [64]:
## parameters for experiment
START = 0
STOP = (LEN_COULOMB_CYCLE / 2 - T_DELAY) / 2
STEPS = 5
NUM_REP = 2


## Experiment

In [65]:
## Define sweep parameter
sweep_length = LinearSweepParameter(
    uid="pulse_length_sweep", start=START, stop=STOP, count=STEPS
)


In [66]:
## Create Experiment
exp = Experiment(
    "Ramsey variant II",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("coulomb_1"),
        ExperimentSignal("coulomb_2"),
    ],
)

# define experiment
with exp.acquire_loop_rt(
    uid="shots", count=NUM_REP, averaging_mode=AveragingMode.SEQUENTIAL
):
    with exp.sweep(uid="length_sweep", parameter=sweep_length):
        with exp.section(uid="qubit_excitation", alignment=SectionAlignment.RIGHT):
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.5)
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.75)

            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.5)
            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.75)

            exp.play(signal="drive", pulse=drive_pulse, length=sweep_length)
            exp.delay(signal="drive", time=T_DELAY)
            exp.play(signal="drive", pulse=drive_pulse, length=sweep_length)
        with exp.section(
            uid="qubit_readout", alignment=SectionAlignment.RIGHT, length=LEN_READOUT
        ):
            exp.play(signal="coulomb_1", pulse=coulomb_readout, amplitude=0.3)
            exp.play(signal="coulomb_2", pulse=coulomb_readout, amplitude=0.3)
            with exp.section(
                uid="outer_trigger",
                length=LEN_READOUT,
                trigger={"drive": {"state": 1}},
                alignment=SectionAlignment.RIGHT,
            ):
                with exp.section(
                    uid="inner_trigger",
                    length=LEN_READOUT - 100e-9,
                    trigger={"drive": {"state": 2}},
                ):
                    exp.reserve(signal="drive")


In [67]:
# define signal maps for different qubits
map_q0 = {
    "drive": "/logical_signal_groups/q0/drive_line",
    "coulomb_1": "/logical_signal_groups/q0/coulomb_line_1",
    "coulomb_2": "/logical_signal_groups/q0/coulomb_line_2",
}

# calibration  for qubit 0
calib_q0 = Calibration()
calib_q0["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=100e6,
        modulation_type=ModulationType.HARDWARE,
    )
)


In [68]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=False)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)

session.run(exp)


2022.12.21 20:10:37.317 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is c:\Users\florianf\OneDrive - Zurich Instruments AG\Documents\QuantumPlaygrounds\florian-playground\LabOneQ_SpinQubits\laboneq_output\log
2022.12.21 20:10:37.318 laboneq.controller.controller  INFO   VERSION: laboneq 1.6.0a20221221135807+0cba0f4d
2022.12.21 20:10:37.320 laboneq.controller.devices.device_collection INFO   Connecting to data server at 10.42.11.2:8004
2022.12.21 20:10:38.497 laboneq.controller.communication INFO   Connected to Zurich Instrument's Data Server version 22.08 at 10.42.11.2:8004
2022.12.21 20:10:50.073 laboneq.controller.devices.device_collection INFO   Configuring clock sources
2022.12.21 20:10:50.174 laboneq.controller.devices.device_collection INFO   Clock sources configured
2022.12.21 20:10:53.805 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2022.12.21 20:10:53.908 l

## View experiment in pulse sheet viewer

In [69]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Ramsey variant II", compiled_exp)


2022.12.21 20:12:11.933 laboneq.pulse_sheet_viewer.pulse_sheet_viewer INFO   Writing html file to c:\Users\florianf\OneDrive - Zurich Instruments AG\Documents\QuantumPlaygrounds\florian-playground\LabOneQ_SpinQubits\Ramsey variant II_2022-12-21-20-12-11.html


c:\Users\florianf\OneDrive - Zurich Instruments AG\Documents\QuantumPlaygrounds\florian-playground\LabOneQ_SpinQubits\Ramsey variant II_2022-12-21-20-12-11.html

# Ramsey Experiment III: 2D sweep, wait time vs phase of second pulse

## Experiment

In [79]:
## constant definition
X90_DURATION = 10e-9  # [s]

START = 0
STOP = 2 * np.pi
STEPS = 5
NUM_REP = 2

START_DELAY = 0
STOP_DELAY = LEN_COULOMB_CYCLE / 2 - 2 * X90_DURATION
STEPS_DELAY = 3


In [80]:
## define phase sweep parameter
sweep_phase = LinearSweepParameter(
    uid="phase_sweep", start=START, stop=STOP, count=STEPS
)
sweep_delay = LinearSweepParameter(
    uid="Ramsey_delay", start=START_DELAY, stop=STOP_DELAY, count=STEPS_DELAY
)

print(sweep_phase.values / np.pi)


[0.  0.5 1.  1.5 2. ]


In [81]:
## Create Experiment
exp = Experiment(
    "Ramsey variant I",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("coulomb_1"),
        ExperimentSignal("coulomb_2"),
    ],
)

# define experiment
with exp.acquire_loop_rt(
    uid="shots", count=NUM_REP, averaging_mode=AveragingMode.CYCLIC
):
    with exp.sweep(uid="sweep_delay", parameter=sweep_delay):
        with exp.sweep(uid="sweep_phase", parameter=sweep_phase):
            with exp.section(
                uid="qubit_excitation",
                alignment=SectionAlignment.RIGHT,
                length=LEN_COULOMB_CYCLE,
            ):
                exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.5)
                exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.75)

                exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.5)
                exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.75)

                exp.play(signal="drive", pulse=drive_pulse, set_oscillator_phase=0)
                exp.delay(signal="drive", time=sweep_delay)
                exp.play(
                    signal="drive",
                    pulse=drive_pulse,
                    increment_oscillator_phase=sweep_phase,
                )
            with exp.section(
                uid="qubit_readout",
                alignment=SectionAlignment.RIGHT,
                length=LEN_READOUT,
            ):
                exp.play(signal="coulomb_1", pulse=coulomb_readout, amplitude=0.3)
                exp.play(signal="coulomb_2", pulse=coulomb_readout, amplitude=0.3)
            with exp.section(
                uid="outer_trigger",
                length=LEN_READOUT,
                trigger={"drive": {"state": 1}},
                alignment=SectionAlignment.RIGHT,
            ):
                with exp.section(
                    uid="inner_trigger",
                    length=LEN_READOUT - 100e-9,
                    trigger={"drive": {"state": 2}},
                ):
                    exp.reserve(signal="drive")


In [82]:
# define signal maps for different qubits
map_q0 = {
    "drive": "/logical_signal_groups/q0/drive_line",
    "coulomb_1": "/logical_signal_groups/q0/coulomb_line_1",
    "coulomb_2": "/logical_signal_groups/q0/coulomb_line_2",
}

# calibration  for qubit 0
calib_q0 = Calibration()
calib_q0["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=0,
        modulation_type=ModulationType.SOFTWARE,
    )
)

print("Set modulation frequency to 0 Hz to better observe the phase sweep.")


Set modulation frequency to 0 Hz to better observe the phase sweep.


In [83]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=False)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)

session.run(exp)


2022.12.21 20:16:18.830 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is c:\Users\florianf\OneDrive - Zurich Instruments AG\Documents\QuantumPlaygrounds\florian-playground\LabOneQ_SpinQubits\laboneq_output\log
2022.12.21 20:16:18.832 laboneq.controller.controller  INFO   VERSION: laboneq 1.6.0a20221221135807+0cba0f4d
2022.12.21 20:16:18.833 laboneq.controller.devices.device_collection INFO   Connecting to data server at 10.42.11.2:8004
2022.12.21 20:16:20.165 laboneq.controller.communication INFO   Connected to Zurich Instrument's Data Server version 22.08 at 10.42.11.2:8004
2022.12.21 20:16:33.142 laboneq.controller.devices.device_collection INFO   Configuring clock sources
2022.12.21 20:16:33.248 laboneq.controller.devices.device_collection INFO   Clock sources configured
2022.12.21 20:16:41.937 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2022.12.21 20:16:42.227 l

## View experiment in pulse sheet viewer

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Ramsey variant III", compiled_exp)
